## Homework_2
## Transfer_Learning
#### Tasks

* Download Cat vs Dogs Kaagle [Competiton](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/overview) dataset
* Print Statistics about counts,labels and shapes, plot random images
* Write data generator(for train, and val), plot results and choose good parameters for data augmantation
* Use Keras VGG-16 pre-trained model, for first froze convolutional base, build and train your classifier part
* Unfroze some layers(Do experiments)
* Try same with ResNet50
* Finally, get more than 95% accuracy

#### Load dataset

In [ ]:
!wget https://www.dropbox.com/s/d61lupw909hc785/dogs_vs_cats.train.zip?dl=1 -O data.zip
!unzip data.zip